In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import pandas as pd
import requests
from io import StringIO
import re
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 
import umap.umap_ as umap
import matplotlib.pyplot as plt
import hdbscan
pd.set_option('display.max_colwidth', None)

In [2]:
sheet_name_lsit = ['Sci/2', 'Sci_Extra/1', 'Sci/1', 'Sci_Extra/2', 'Engineering/1', 'Engineering_Extra/1', 'Engineering/2', 'Engineering_Extra/2', 'SIIT/2', 'SIIT/1', 'Innovation_Extra/1', 'Innovation_Extra/2', 'CIS_Extra/1', 'CIS/1', 'CIS_Extra/2', 'CIS/2']

In [3]:
sheet_id = "1ExtEUwVLi5Y_mmqxNd0oX7WrWzj8n6TeBLcQg7Xu0GU"
dataframe = pd.DataFrame(columns=["code", "description", "disciplinary course(s)", "consecutive course(s)", "add3","faculty"])

for sheet_name in sheet_name_lsit:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    response = requests.get(url)
    decoded_content = response.content.decode('utf-8')
    data_tem =  pd.read_csv(StringIO(decoded_content))
    dataframe = pd.concat([dataframe, data_tem], ignore_index=True)

print(dataframe.shape)
for i in dataframe.columns:
    dataframe[f"{i}"] = dataframe[f"{i}"].str.strip().str.lower()


print(dataframe.shape)

(4272, 6)
(4272, 6)


In [4]:
dataframe["description"] = dataframe["description"].str.replace(r"[.,]", "", regex=True).str.strip()

dataframe.drop_duplicates(subset=["code"],inplace=True)
dataframe.dropna(subset=["description","code"],inplace=True)


In [5]:
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คพ","cs")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ทก","at")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คป","am")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ทช","bt")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("วท","sc")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คม","cm")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ค","ma")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ส","st")

dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คพ","cs")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ทก","at")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คป","am")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ทช","bt")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("วท","sc")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คม","cm")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ค","ma")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ส","st")


In [6]:
def extract_courses(row):
    pattern = r"[A-Za-zก-ฮ]{1,2}\.? ?\d{3}"
    listt = []
    if isinstance(row, str):
        matches = re.findall(pattern, row)  
        listt.extend(matches)
        return listt
    return None 

dataframe["cancel_course"] = dataframe["disciplinary course(s)"].apply(extract_courses) + dataframe["consecutive course(s)"].apply(extract_courses)

In [7]:
dataframe.drop("add3",inplace=True,axis=1)

In [10]:
dataframe.to_csv("data1.csv")